# Computations of emissions from our business markets customers
* Include emissions from real estate only (not deposits or project finance)

In [9]:
import pandas as pd
import numpy as np

In [35]:
# The absolute file path may need to be updated.
abs_file_path = '...'

df_tenants = pd.read_excel(abs_file_path+"mortgageData.xlsx")
df_tenants = df_tenants[["Leietaker", "KvmUtleid", "Standard", "ArealType", "LTV_Intern" , "Byggeår", "Energikarakter", "Oppvarmingskarakter", "Naeringskode", "Naeringsnavn", "Naeringsnavn2", "Naeringsnavn3", "Naeringsnavn4", "Naeringsnavn5"]]


,Leietaker,KvmUtleid,Standard,ArealType,LTV_Intern,Byggeår,Energikarakter,Oppvarmingskarakter,Naeringskode,Naeringsnavn,Naeringsnavn2,Naeringsnavn3,Naeringsnavn4,Naeringsnavn5


In [42]:

# Some areas and tenants are not relevant, eg. outdoor parking, sold or empty apartments, space listed as unused or common. These are removed.
# Operator ~ inverts boolean values.
df_tenants = df_tenants[(df_tenants["Leietaker"] != "INNBETALING")]
df_tenants = df_tenants[~(df_tenants["Leietaker"].str.lower().str.contains("fiktiv"))]
df_tenants = df_tenants[~((df_tenants["ArealType"] == "Hotell") & (df_tenants["Leietaker"].str.lower().str.contains("park")))]
df_tenants = df_tenants[~(df_tenants["Leietaker"].str.contains("Solgt"))]
df_tenants.reset_index(drop=True, inplace=True)

# Fix unlogical LTV values
df_tenants["LTV_Intern"].fillna(1, inplace=True)
df_tenants.loc[df_tenants['LTV_Intern'] > 1, "LTV_Intern"] = 1

## Specifying building type
The data contains data on building types, but groups only in Lager, Bolig, Kontor, Ukjent, Butikk, Hotell.
The goal is to specify such that the types are the same as those in the energy grade system.

In [1]:
# df_tenants.head(20)

In [44]:
# Kindergardens
df_tenants.loc[df_tenants["Leietaker"].str.lower().str.contains("barneha") | (df_tenants["Naeringsnavn5"] == "Barnehager"), "ArealType"] = "Barnehage"

# Schools, included trafic schools
df_tenants.loc[df_tenants["Leietaker"].str.lower().str.contains("skole"), "ArealType"] = "Skolebygning"

# Hospitals
df_tenants.loc[df_tenants["Leietaker"].str.lower().str.contains("sykehus") | df_tenants["Naeringsnavn5"].str.lower().str.contains("Trenin"), "ArealType"] = "Sykehus"

# Nurseryhomes. Industrycodes for hospital services: 87.302
df_tenants.loc[df_tenants["Naeringskode"] == 87.302, "ArealType"] = "Sykehjem"

# Hotels. A few restauratns cafees and parking station are included
df_tenants.loc[df_tenants["Leietaker"].str.lower().str.contains("hotel") | (df_tenants["ArealType"] == "Hotell") , "ArealType"] = "Hotellbygning"

# Athletics facilities
df_tenants.loc[df_tenants["Leietaker"].str.lower().str.contains("trening") | (df_tenants["Naeringsnavn5"] == "Treningssentre") | (df_tenants["Naeringsnavn5"] == "Idrettslag og -klubber"), "ArealType"] = "Idrettsbygning"

# Grocery store
df_tenants.loc[(df_tenants["Leietaker"].str.lower().str[:4] == "rema") | (df_tenants["Leietaker"].str.lower().str[:4] == "kiwi"), "ArealType"] = "Dagligvare"
df_tenants.loc[df_tenants["Leietaker"].str.lower().str.contains("bunnpris") | df_tenants["Leietaker"].str.lower().str.contains("meny"), "ArealType"] = "Dagligvare"

df_tenants[df_tenants.ArealType == "Dagligvare"]

# Commercial buildings 
df_tenants.loc[(df_tenants["ArealType"] == "Butikk") , "ArealType"] = "Forretningsbygning"

# Light manufacturing, included workshops
## Includes wharehouses
df_tenants.loc[df_tenants["Leietaker"].str.lower().str.contains("verksted") , "ArealType"] = "Lett industribygning, verksted"
df_tenants.loc[(df_tenants["Naeringsnavn2"] == "Handel med og reparasjon av motorvogner"), "ArealType"] = "Lett industribygning, verksted"
df_tenants.loc[df_tenants["ArealType"] == "Lager", "ArealType"]  = "Lett industribygning, verksted"

# Offices
df_tenants.loc[df_tenants["ArealType"] == "Kontor", "ArealType"] = "Kontorbygning" 

# Mortgages
# Apartments and small houses
df_tenants.loc[(df_tenants["ArealType"] == "Bolig") & (df_tenants["KvmUtleid"] < 120), "ArealType"]  = "Leiligheter (boligblokk)"
df_tenants.loc[(df_tenants["ArealType"] == "Bolig") & (df_tenants["KvmUtleid"] >= 120), "ArealType"]  = "Småhus"

# Vacant offices
# df_tenants = df_tenants[(df_tenants["Leietaker"] != "INNBETALING") & ~(df_tenants["Leietaker"].str.lower().str.contains("ledig")) & ~(df_tenants["Leietaker"].str.lower().str.contains("felles"))]


## Energy grade system
For each grade and type of building, the mumbers indicate the hmaximum energy consumption in kWh per square meter per year.
Added building type _Dagligvare_ from a report for [energimerking.no by Sintef](https://www.energimerking.no/download?objectPath=/upload_images%2F1D6EE42046FB4DA78BFCFA9470EA5607.pdf), p. 44

Our buiding types:
Lager, Bolig, Kontor, Ukjent, Butikk, Hotell

In [46]:
url = "https://www.energimerking.no/download?objectPath=/upload_images%2FFC1EA4B47E3141AFAD2CFD75C7829E6F.xlsx"

df_energyGrade = pd.read_excel(url, skiprows=4, skipfooter=43)
df_energyGrade = df_energyGrade.drop(0).rename(columns={'Unnamed: 0': "Type"}).set_index("Type", drop=True)
df_energyGrade.loc["Dagligvare"] = [265, 400, 530, 605, 680, 1020, "> F"]
df_energyGrade

,A,B,C,D,E,F,G
Type,,,,,,,
Småhus,95,120,145,175,205,250,>F
Arealkorreksjon,+800/A,+1600/A,+2500/A,+4100/A,+5800/A,+8000/A,NaN
Leiligheter (boligblokk),85,95,110,135,160,200,>F
Arealkorreksjon,+600/A,+1000/A,+1500/A,+2200/A,+3000/A,+4000/A,NaN
Barnehage,85,115,145,180,220,275,> F
Kontorbygning,90,115,145,180,220,275,> F
Skolebygning,75,105,135,175,220,280,> F
Universitets- og høgskolebygning,90,125,160,200,240,300,> F
Sykehus,175,240,305,360,415,505,> F


## Energy consumption regulations based on building/renovation year
Data taken from Requirements for the Planning and Building Act (Forskrifter til plan- og bygningsloven, *TEK*), which in 2007 set regulations on net energy consumption per square meter per year for buildings. The dataframe contains the newest regulations on energy consumption.

Source: https://www.regjeringen.no/no/tema/plan-bygg-og-eiendom/bygningsregelverket-fra-1965--20172/forskrifter/id2590708/

In [47]:
df_buildingYear = pd.DataFrame(index=["Småhus", "Leiligheter (boligblokk)", "Barnehage", "Kontorbygning", "Skolebygning", "Universitets- og høgskolebygning", "Sykehus", "Sykehjem", "Hotellbygning", "Idrettsbygning", "Forretningsbygning", "Kulturbygning", "Lett industribygning, verksted", "Dagligvare"], columns=[2007, 2010, 2016])
df_buildingYear[2007] = [125, 120, 150, 165, 135, 180, 325, 235, 240, 185, 235, 180, 185, 680]
df_buildingYear[2010] = [120, 115, 140, 150, 120, 160, 300, 215, 220, 170, 210, 165, 175, 520]
df_buildingYear[2016] = [100, 95, 135, 115, 110, 125, 225, 195, 170, 145, 180, 130, 140, 400]

df_buildingYear

,2007,2010,2016
Småhus,125,120,100
Leiligheter (boligblokk),120,115,95
Barnehage,150,140,135
Kontorbygning,165,150,115
Skolebygning,135,120,110
Universitets- og høgskolebygning,180,160,125
Sykehus,325,300,225
Sykehjem,235,215,195
Hotellbygning,240,220,170
Idrettsbygning,185,170,145


## Average net energy consumption
If no data on energy consumption is available for a given tenant, average data for the building type is used. 
Data is from [Enovas byggstatistikk 2017](https://www.enova.no/download?objectPath=upload_images/5C6245BC2AD74248BB629BFA95145AA3.pdf&filename=Enovas%20byggstatistikk%202017.pdf), page 20, fig. 4-7.

In [48]:
averageConsumption = pd.Series([166, 143, 181, 540, 179, 140, 261, 310, 224, 272, 237, 235, 190, 269], 
                                  index=["Småhus", "Leiligheter (boligblokk)", "Barnehage", "Dagligvare", "Kontorbygning", "Skolebygning", "Universitets- og høgskolebygning", "Sykehus", "Sykehjem", "Hotellbygning", "Idrettsbygning", "Forretningsbygning", "Kulturbygning", "Lett industribygning, verksted"], dtype=int)
averageConsumption

Småhus                              166
Leiligheter (boligblokk)            143
Barnehage                           181
Dagligvare                          540
Kontorbygning                       179
Skolebygning                        140
Universitets- og høgskolebygning    261
Sykehus                             310
Sykehjem                            224
Hotellbygning                       272
Idrettsbygning                      237
Forretningsbygning                  235
Kulturbygning                       190
Lett industribygning, verksted      269
dtype: int32

## Calculations
As the tenant LTV is most often not 100%, it is used as a measure of how much of the rented area which is part of our portfolio.
Until data on tenant energy grades is available, we can use the parameter _standard_ to estimate the energy grade.  According to TEK10 (teknisk forskrift), a modern building in good standard will satisfy the requirements for energy grade C. 

For enegy grade _G_ we use numbers for grade _F_

In [65]:
def net_energy_consuption(arealType, kvm, std, LTV, energyGrade, buildingYear):
    cols_reverse = list(df_buildingYear.columns)
    cols_reverse.reverse()
    
    # Use energy grade
    if(isinstance(energyGrade, str)):
        if energyGrade == "G": energyGrade = 'F'
        kWh = df_energyGrade.loc[arealType, energyGrade] * kvm * LTV
        
    # Use building year
    # If the year is lower than 2007, use 'standard' parameter
    elif(buildingYear):
        for year in cols_reverse:
            if(buildingYear > year):
                kWh = df_buildingYear.loc[arealType, year] * kvm * LTV
            else: 
                energyGrade = {"God": "C", "Middels": "D", "Dårlig": "F", "Ukjent": "F"}[std]
                kWh = df_energyGrade.loc[arealType, energyGrade] * kvm * LTV
    
    # Use average data
    #else:
    #    kWh = averageConsumption[arealType] * kvm * LTV
    
    # Use building standard as 'energy grade'
    else: 
        energyGrade = {"God": "C", "Middels": "D", "Dårlig": "F", "Ukjent": "F"}[std]
        kWh = df_energyGrade.loc[arealType, energyGrade] * kvm * LTV
        
    return kWh

In [2]:
# df_tenants.head(10)


In [1]:
# df_tenants["net_energy_consumtpion"] = df.apply (lambda row: net_energy_consuption(row.ArealType, row.KvmUtleid, row.Standard, row.LTV_Intern), axis=1)
# df_tenants
lst = []
for index, row in df_tenants.iterrows():
    tenant_kWh = net_energy_consuption(df_tenants.loc[index, "ArealType"], df_tenants.loc[index, "KvmUtleid"], df_tenants.loc[index, "Standard"], df_tenants.loc[index, "LTV_Intern"], df_tenants.loc[index, "Energikarakter"], df_tenants.loc[index, "Byggeår"])
    lst.append(tenant_kWh)
    
df_tenants["net_energy_consumtpion"] = lst

## Summary

In [2]:
df_tenants_grouped = df_tenants.groupby(['ArealType']).sum()[["KvmUtleid", "net_energy_consumtpion"]]

df_tenants_grouped.rename(columns={"net_energy_consumtpion": "net_energy_consumtpion_GWh"}, inplace=True)
df_tenants_grouped["net_energy_consumtpion_GWh"] = df_tenants_grouped["net_energy_consumtpion_GWh"] / 1000000

# Group ArealTypes
df_tenants_grouped.loc["Privatboliger"] = df_tenants_grouped.loc["Leiligheter (boligblokk)"]  + df_tenants_grouped.loc["Småhus"]
df_tenants_grouped.drop(["Leiligheter (boligblokk)", "Småhus"], inplace=True)

df_tenants_grouped.loc["Skole og barnehage"] = df_tenants_grouped.loc["Skolebygning"]  + df_tenants_grouped.loc["Barnehage"]
df_tenants_grouped.drop(["Skolebygning", "Barnehage"], inplace=True)

df_tenants_grouped.loc["Sykehjem og sykehus"] = df_tenants_grouped.loc["Sykehjem"]
df_tenants_grouped.drop(["Sykehjem"], inplace=True)

In [3]:
print("Consumed energy from comercial buildings (Næringsbygg), by building type: ")
print("df_tenants_grouped: ")
display(df_tenants_grouped)